In [1]:
# project: p10
# submitter: jstump
# partner: njannusch
# hours: 5

In [2]:
import os
import csv
import json
from collections import namedtuple

In [3]:
def list_files_in(pathname):
    data_files = os.listdir(pathname)
    data_files = sorted(data_files)
    data_files_filtered = [data_files[i] for i in range(len(data_files)) if data_files[i][0] != '.']
    return data_files_filtered



def list_paths_in(pathname):
    files_list = list_files_in(pathname)
    paths_list = [os.path.join(pathname, files_list[i]) for i in range(len(files_list))]
    return paths_list



In [4]:
#Q1: What are the names of the files present in the data directory?

list_files_in('data')

['channel_ids1.json',
 'channel_ids2.json',
 'channel_ids3.json',
 'channel_ids4.json',
 'channel_ids5.json',
 'comment_data1.csv',
 'comment_data2.csv',
 'comment_data3.csv',
 'comment_data4.csv',
 'comment_data5.csv',
 'video_data.csv',
 'video_ids.json']

In [5]:
#Q2: What are the paths of all the files in the data directory?

paths_data = list_paths_in('data')
paths_data

['data/channel_ids1.json',
 'data/channel_ids2.json',
 'data/channel_ids3.json',
 'data/channel_ids4.json',
 'data/channel_ids5.json',
 'data/comment_data1.csv',
 'data/comment_data2.csv',
 'data/comment_data3.csv',
 'data/comment_data4.csv',
 'data/comment_data5.csv',
 'data/video_data.csv',
 'data/video_ids.json']

In [6]:
#Q3: What are the paths of all the files whose name contains comment_data in our data directory?
comment_paths = [item for item in paths_data if 'comment_data' in item]
comment_paths


['data/comment_data1.csv',
 'data/comment_data2.csv',
 'data/comment_data3.csv',
 'data/comment_data4.csv',
 'data/comment_data5.csv']

In [7]:
#Q4: What are the paths of all the files whose name contains channel_ids in our data directory?

channel_paths = [item for item in paths_data if 'channel_ids' in item]
channel_paths

['data/channel_ids1.json',
 'data/channel_ids2.json',
 'data/channel_ids3.json',
 'data/channel_ids4.json',
 'data/channel_ids5.json']

In [8]:
def read_json(path):
    with open(path, encoding="utf-8") as f:
        return json.load(f) # dict, list, etc


def get_mapping(pathname):
    try:
        json_data = read_json(pathname)
        return json_data
    except(json.JSONDecodeError):
        return {}

In [9]:
#Q5: What is the name of the channel with the ID UCNye-wNBqNL5ZzHSJj3l8Bg?

json_data = get_mapping(os.path.join('data', 'channel_ids1.json'))
json_data['UCNye-wNBqNL5ZzHSJj3l8Bg']

'Al Jazeera English'

In [10]:
#Q6: How many channels do we have in the entire dataset?
channel_dict = {}
for path in channel_paths:
    for key in get_mapping(path):
        channel_dict[key] = get_mapping(path)[key]
len(channel_dict)

    


372

In [11]:
Comment = namedtuple('Comment', ['video_id','comment_length','author_id','likes','published_at'])
   
def process_csv(filename):
    exampleFile = open(filename, encoding="utf-8")  
    exampleReader = csv.reader(exampleFile)
    exampleData = list(exampleReader)        
    exampleFile.close()  
    return exampleData

def get_comment_data(comment_file):
    csv_data = process_csv(comment_file)
    header = csv_data[0]
   
    # TODO: if required, use a print function call to display header or alternatively, take a look at the comments csv file
    comment_rows = csv_data[1:]
    comment_id_idx = header.index("comment_id")
    video_id_idx = header.index("video_id")
    comment_length_idx = header.index('comment_length')
    author_id_idx = header.index('author_id')
    likes_idx = header.index('likes')
    published_at_idx = header.index('published_at')
   
    comment_dict = {}
   
    for comment in comment_rows:
        try:
            comment_id = comment[comment_id_idx]
            video_id = comment[video_id_idx]
            author_id = comment[author_id_idx]
            published_at = comment[published_at_idx]
            likes = comment[likes_idx]
            comment_length = comment[comment_length_idx]
            if comment_id == '' or video_id == '' or author_id == '' or published_at == '' or likes == '' or comment_length == '':
                continue
            likes = int(comment[likes_idx])
            comment_length = int(comment[comment_length_idx])
        except(IndexError,ValueError):
            continue
           
        comment_id = comment[comment_id_idx]
        video_id = comment[video_id_idx]
        author_id = comment[author_id_idx]
        published_at = comment[published_at_idx]
     
        current_comment = Comment(video_id,comment_length,author_id,likes,published_at)
       
        comment_dict[comment_id] = current_comment
    return comment_dict

In [12]:
#Q7: What is the Comment object with comment ID UgygOezB4Mvd5o6FgAt4AaABAg?
get_comment_data(os.path.join('data', 'comment_data1.csv'))['UgygOezB4Mvd5o6FgAt4AaABAg']

Comment(video_id='udNXMAflbU8', comment_length=175, author_id='UCHkk7x38KWgqjQOHqsQwf0Q', likes=47, published_at='2021-10-10 17:48:38')

In [13]:
comments = {}
for file in comment_paths:
    comments.update(get_comment_data(file))

In [14]:
#Q8: What is the length of the comment with ID UgztIaGfqFoiGvbOdfp4AaABAg?
comments['UgztIaGfqFoiGvbOdfp4AaABAg'].comment_length

67

In [15]:
#Q9: What percentage of comments are at most 140 characters long?
count = 0
total = 0
for key in comments:
    total +=1
    if comments[key].comment_length <= 140:
        count +=1
count / total * 100

86.86953042956443

In [16]:
#Q10: What is the author ID of the comment that has the highest number of likes?
highest_number = 0
highest_ID = None
for key in comments:
    if comments[key].likes > highest_number:
        highest_number = comments[key].likes
        highest_ID = comments[key].author_id
highest_ID

'UCIPPMRA040LQr5QPyJEbmXA'

In [17]:
#Q11: What is the most popular hour for publishing comments?
comment_hr_dict = {}
for comment_id in comments:
    current_hour = comments[comment_id].published_at[-8:-6]
    if current_hour not in comment_hr_dict:
        comment_hr_dict[current_hour] = 0
    comment_hr_dict[current_hour] += 1
   
most_hr = None
most_count = 0
for hour in comment_hr_dict:
    if comment_hr_dict[hour] > most_count:
        most_count = comment_hr_dict[hour]
        most_hour = hour
int(most_hour)


19

In [18]:
#Q12: How many comments does the video with ID A8rrr_w8rfk have?
comment_buckets = {}
for comment_id in comments:
    if comments[comment_id].video_id not in comment_buckets:
        comment_buckets[comments[comment_id].video_id] = []
    comment_buckets[comments[comment_id].video_id].append(comment_id)
len(comment_buckets['A8rrr_w8rfk'])

606

In [19]:
def get_videos(data_file, video_mapping_file):
    """
    Given data_file (csv file) and video_mapping_file (json) file, generates a video
    dictionary, mapping video ID to a dictionary containing title, and other details of the video.
    Handles missing entry in video_ids.json by using try / except blocks to handle KeyError.
    """
    data = process_csv(data_file)
    header = data[0]
    all_videos = data[1:]
    video_mapping = get_mapping(video_mapping_file)
    videos_dict = {}
    for i in range(len(all_videos)): # You may find it helpful to do all_videos[:5] to only look at the first 5 videos.
        video_id = all_videos[i][header.index('video_id')]
        published = all_videos[i][header.index('published_at')]
        duration = all_videos[i][header.index('duration')]
        category = all_videos[i][header.index('category')]
        views = all_videos[i][header.index('views')]
        tags = all_videos[i][header.index('tags')].split('|')
        likes = all_videos[i][header.index('likes')]
        dislikes = all_videos[i][header.index('dislikes')]
        channel_id = all_videos[i][header.index('channel_id')]
        if likes == '':
            likes = None
        else:
            likes = int(likes)
        if dislikes == '':
            dislikes = None
        else:
             dislikes = int(dislikes)   
        if dislikes == None and likes == None:
            ratings_enabled = False
        else:
            ratings_enabled = True
        
        inner_dict = {}
        try:
            inner_dict['title'] = video_mapping[video_id]
            inner_dict['published_at'] = published
            inner_dict['duration'] = duration
            inner_dict['category'] = category
            inner_dict['views'] = int(views)  
            inner_dict['tags'] = tags
            inner_dict['likes'] = likes
            inner_dict['dislikes'] = dislikes
            inner_dict['ratings_enabled'] = ratings_enabled
            inner_dict['channel_name'] = channel_dict[channel_id]
            inner_dict['comments'] = comment_buckets[video_id]
        except(KeyError):
            continue
        try:
            videos_dict[video_id] = inner_dict
        except(KeyError):
            continue
             
    
        # TODO: for start, use print function call to display each video row
        # TODO: Construct the dictionary.
    return videos_dict

videos = get_videos(os.path.join('data','video_data.csv'), os.path.join('data','video_ids.json'))



In [20]:
sample = ['UgzgwN2JXxjTN4mR5954AaABAg.9TPxukUd20g9TQLnJi3RFU', 'UgzvogxMg82Kj0aW84x4AaABAg']
for s in sample:
    assert s in videos['fkMW60W180E']['comments']

In [21]:
#Q13: What is the video with ID fkMW60W180E?
videos['fkMW60W180E']

{'title': 'SWAWS | Totally Accurate Battlegrounds',
 'published_at': '2021-10-12 19:01:41',
 'duration': '00:18:46',
 'category': 'Gaming',
 'views': 3172185,
 'tags': ['tot',
  'totally accurate battlegrounds',
  'tabg',
  'totally accurate battle simulator',
  'totally accurate battlegrounds gameplay',
  'totally accurate battle grounds',
  'tabg gameplay',
  'tabg game',
  'tabs',
  'totally accurate',
  'totally accurate battle royale',
  'battle royale',
  'tabg funny',
  'fortnite',
  'battlegrounds',
  'tabs battle royale',
  'new battle royale',
  'pubg',
  'totally accurate battlegrounds funny',
  'swaws',
  'swaws meme',
  'swaws russian badger',
  'tabg update',
  'tabg win',
  'tabg br',
  'free to play pc games',
  'free to play'],
 'likes': 210951,
 'dislikes': 1824,
 'ratings_enabled': True,
 'channel_name': 'TheRussianBadger',
 'comments': ['UgzgwN2JXxjTN4mR5954AaABAg.9TPxukUd20g9TQLnJi3RFU',
  'Ugw8z3sbkSQQpgINHdl4AaABAg',
  'UgyjZAl0XlNg-gOZ0jR4AaABAg.9TPyCmI6pbl9TQAX

In [22]:
#Q14: What is the title of the video with ID gF69voHU_ys?
videos['gF69voHU_ys']['title']

'A Mukbanger Ate 1 Gallon Pickles. This Is What Happened To Her Brain.'

In [23]:
#Q15: Among the videos with more than 1 million views, what is the title of the video with the highest likes to views ratio?
max_ratio = 0
max_name = None
for key in videos:
    views_videos = videos[key]['views']
    likes_videos = videos[key]['likes']
    if views_videos > 1000000:
        if likes_videos != None:
            ratio = likes_videos / views_videos
            if ratio > max_ratio:
                max_ratio = ratio
                max_name = videos[key]['title']
max_name
        
        

'[In the SOOP BTS ver. Season 2] Official Teaser 2'

In [24]:
#Q16: What is the author ID of the most liked comment under the video titled 'Giving Away My Beard For Charity!'

most_likes = 0
most_liked_author = None
for video_id in videos:
    if videos[video_id]['title'] == 'Giving Away My Beard For Charity!':
        for comment_id in videos[video_id]['comments']:
            if comments[comment_id].likes > most_likes:
                most_likes = comments[comment_id].likes
                most_liked_author = comments[comment_id].author_id
most_liked_author

'UCRvcDpyxCVmlLI8Q2TjRpmg'

In [25]:
type(videos['gF69voHU_ys']['channel_name'])

str

In [26]:
def bucketize(attribute, videos = videos):
    #specific_movie_list = find_specific_movies(movie_list, category)
    videos_dict = {}
    if attribute == 'title' or attribute == 'published_at' or attribute == 'duration' or attribute == 'category' or attribute == 'views'\
    or attribute == 'tags' or attribute == 'likes' or attribute == 'dislikes' or attribute == 'ratings_enabled' or attribute == 'channel_name'\
    or attribute == 'comments':
        for key in videos:
            item = videos[key][attribute]
            if type(item) == list:
                for i in range(len(item)):
                    if item[i] not in videos_dict:
                        videos_dict[item[i]] = []
                    if item[i] in videos_dict:
                        videos_dict[item[i]].append(key)    
            elif type(item) != list:
                if item not in videos_dict:
                    videos_dict[item] = []
                if item in videos_dict:
                    videos_dict[item].append(key)
        return videos_dict

In [27]:
category_dict = bucketize('category')
assert category_dict['Pets & Animals'] == ['Hz_DslzN2IA', 'AwvyrO_yM4c']

In [28]:
#Q17: Which video titles were produced by the "Corridor Crew" channel?
channel_buckets = bucketize('channel_name')
video_id_crew = channel_buckets["Corridor Crew"]
titles_list = []
for item in video_id_crew:
    titles_list.append(videos[item]['title'])
titles_list



['VFX Artists React to SHANG-CHi Bad & Great CGi',
 'VFX Artists React to Bad & Great CGi 54 (ft. SETH ROGEN)']

In [29]:
#Q18: What are the top 5 channels that have the most total comments on their videos?
total_comments = {}
for key in channel_buckets:
    total = 0
    for i in range(len(channel_buckets[key])):
        total += len(videos[channel_buckets[key][i]]['comments'])
    total_comments[key] = total
sorted_dict = dict(sorted(total_comments.items(), key = lambda item: item[1], reverse = True))
list(sorted_dict)[:5]

  
  

        

['NFL', 'Minecraft', 'Saturday Night Live', 'SSSniperWolf', 'PBC ON FOX']

In [30]:
#Q19: List all the unique video titles which have Minecraft tags.
minecraft_videos = []
tags_buckets = bucketize('tags', videos)
for tag in tags_buckets:
    if 'minecraft' in tag.lower():
        for i in range(len(tags_buckets[tag])):
            minecraft_videos.append(tags_buckets[tag][i-1])
minecraft_titles = []
for video_id in minecraft_videos:
    minecraft_titles.append(videos[video_id]['title'])
list(set(minecraft_titles))
    


['I Designed Custom Minecraft Bosses...',
 'can 1 pixel of water save you?',
 "Minecraft, But We're All Mutant Superhumans...",
 'Which Mob Am I Voting For in Minecraft Live 2021?!',
 'Building AUTOMATIC FARMS in Minecraft Hardcore (#52)',
 'You Laugh You Lose Is Even Funnier In Real Life...',
 'Realistic Water in Minecraft...',
 'I Survived 69 Hours Using Only Minecraft Tools',
 '46 Minecraft Blocks Mojang Had To Remove',
 'Playing SQUID GAME In Minecraft!',
 'Minecraft Mobs if they were Superheroes',
 'Minecraft Death Shuffle...',
 'We Fought a Viking Horde...',
 'Playing SQUID GAME in Minecraft!',
 'Last Life: Episode 4 - UH OH!',
 'Minecraft but you can Eat Mobs',
 'So I made Dance Moves control Minecraft...',
 'Jumping SQUID GAME CARS Across ENTIRE MAP In GTA 5! (Mods)',
 'Minecraft, But You Can Upgrade TNT...',
 'What is the Safest Base Possible in Minecraft?',
 'I Survived 100 Days as a PANDA in HARDCORE Minecraft!',
 'Minecraft Manhunt but I have CUSTOM COMMANDS',
 "Minecraft, 

In [31]:
#Q20: List the titles of the 5 shortest videos.
duration_dict = {}
for key in videos:
    duration = videos[key]['duration']
    duration_dict[videos[key]['title']] = duration
sorted_dict = dict(sorted(duration_dict.items(), key = lambda d:d[1]))
list(sorted_dict)[:5]
    
    
    
    
    

['When someone walks in while you using the bathroom😭',
 'Smart Appliances, Gadgets For Every Home 🤩 Versatile Utensils, Kitchen, Makeup, Beauty #Shorts',
 'Italians reacting to ketchup on pasta 😱🇮🇹 #shorts',
 'My hidden talent #shorts',
 'Knot']